In [ ]:
# Unzip and load the dataset
import zipfile
import pandas as pd

print("Extracting dataset...")
with zipfile.ZipFile('PS_20174392719_1491204439457_log.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print("✓ Extracted!")

print("\nLoading dataset...")
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

print(f"✓ Dataset loaded: {df.shape}")
print(f"✓ Fraud: {df['isFraud'].sum():,} ({df['isFraud'].mean()*100:.4f}%)")
print("\n✓ Ready to run pipeline!")

df.head(3)

In [ ]:
# ============================================================================
# COMPLETE FRAUD DETECTION PROJECT
# ============================================================================
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE
import shap
import plotly.express as px

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("="*80)
print("FRAUD DETECTION - COMPLETE PIPELINE")
print("="*80)
print(f"\nDataset: {len(df):,} transactions | Fraud: {df['isFraud'].sum():,} ({df['isFraud'].mean()*100:.4f}%)")

# ============================================================================
# [1/6] FEATURE ENGINEERING
# ============================================================================
print("\n[1/6] Feature Engineering...")
df_feat = df.copy()

df_feat['amount_log'] = np.log1p(df_feat['amount'])
df_feat['amount_sqrt'] = np.sqrt(df_feat['amount'])
df_feat['orig_balance_ratio'] = df_feat['amount'] / (df_feat['oldbalanceOrg'] + 1)
df_feat['dest_balance_ratio'] = df_feat['amount'] / (df_feat['oldbalanceDest'] + 1)
df_feat['is_amount_mismatch'] = (np.abs(df_feat['amount'] - (df_feat['oldbalanceOrg'] - df_feat['newbalanceOrig'])) > 0.01).astype(int)
df_feat['is_orig_zero'] = (df_feat['oldbalanceOrg'] == 0).astype(int)
df_feat['is_dest_zero'] = (df_feat['oldbalanceDest'] == 0).astype(int)
df_feat['is_orig_drained'] = (df_feat['newbalanceOrig'] == 0).astype(int)
df_feat['hour'] = df_feat['step'] % 24
df_feat['day'] = df_feat['step'] // 24
df_feat['type_encoded'] = LabelEncoder().fit_transform(df_feat['type'])
type_dummies = pd.get_dummies(df_feat['type'], prefix='type')
df_feat = pd.concat([df_feat, type_dummies], axis=1)
df_feat['is_dest_merchant'] = df_feat['nameDest'].str.startswith('M').astype(int)
df_feat['is_orig_merchant'] = df_feat['nameOrig'].str.startswith('M').astype(int)
df_feat['amount_pct'] = df_feat.groupby('type')['amount'].transform(lambda x: x.rank(pct=True))

print(f"   ✓ Created {df_feat.shape[1] - df.shape[1]} new features")

# ============================================================================
# [2/6] PREPROCESSING
# ============================================================================
print("\n[2/6] Preprocessing...")
df_proc = df_feat.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud', 'type'], errors='ignore')
X = df_proc.drop('isFraud', axis=1)
y = df_proc['isFraud']
X = X.replace([np.inf, -np.inf], np.nan).fillna(X.median())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print(f"   ✓ Features: {X.shape[1]} | Train: {len(X_train):,} | Test: {len(X_test):,}")

# ============================================================================
# [3/6] SMOTE
# ============================================================================
print("\n[3/6] Applying SMOTE (1-2 minutes)...")
smote = SMOTE(sampling_strategy=0.5, random_state=RANDOM_STATE)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
print(f"   ✓ Balanced: {len(X_train_balanced):,} samples")

# ============================================================================
# [4/6] TRAIN MODELS
# ============================================================================
print("\n[4/6] Training Models...")
models = {
    'XGBoost': XGBClassifier(n_estimators=100, max_depth=6, random_state=RANDOM_STATE, eval_metric='logloss', verbosity=0),
    'LightGBM': LGBMClassifier(n_estimators=100, max_depth=6, random_state=RANDOM_STATE, verbosity=-1),
    'CatBoost': CatBoostClassifier(iterations=100, depth=6, random_state=RANDOM_STATE, verbose=0)
}

for name, model in models.items():
    model.fit(X_train_balanced, y_train_balanced)
    print(f"   ✓ {name}")

# ============================================================================
# [5/6] EVALUATE
# ============================================================================
print("\n[5/6] Evaluating...")
results = []
predictions = {}

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba)
    })
    predictions[name] = {'pred': y_pred, 'proba': y_proba}

results_df = pd.DataFrame(results).sort_values('ROC-AUC', ascending=False)
best_name = results_df.iloc[0]['Model']
best_model = models[best_name]

print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)
print(results_df.to_string(index=False))
print(f"\nBest: {best_name} (ROC-AUC: {results_df.iloc[0]['ROC-AUC']:.4f})")

# Visualize
fig = px.bar(results_df, x='Model', y=['Precision', 'Recall', 'F1-Score', 'ROC-AUC'],
             title='Performance Comparison', barmode='group', height=500)
fig.show()

# Confusion Matrix
cm = confusion_matrix(y_test, predictions[best_name]['pred'])
print(f"\nConfusion Matrix:\n  TP: {cm[1,1]:,} | FN: {cm[1,0]:,}\n  FP: {cm[0,1]:,} | TN: {cm[0,0]:,}")

# ============================================================================
# [6/6] SHAP
# ============================================================================
print("\n[6/6] SHAP Analysis...")
X_sample = X_train_balanced.sample(n=1000, random_state=RANDOM_STATE)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_sample)
if isinstance(shap_values, list):
    shap_values = shap_values[1]

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False, max_display=10)
plt.title(f'Top Features - {best_name}')
plt.tight_layout()
plt.show()

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*80)
print("PROJECT COMPLETE")
print("="*80)
print(f"""
Dataset: {len(df):,} transactions
Best Model: {best_name}
ROC-AUC: {results_df.iloc[0]['ROC-AUC']:.4f}
Precision: {results_df.iloc[0]['Precision']:.4f}
Recall: {results_df.iloc[0]['Recall']:.4f}

Status: COMPLETE
""")
print("="*80)

In [ ]:
# ============================================================================
# SAVE MODELS AND ARTIFACTS
# ============================================================================
import pickle
import json
from datetime import datetime

print("="*80)
print("SAVING MODELS AND ARTIFACTS")
print("="*80)

# Create output directory
!mkdir -p fraud_detection_models

# 1. Save all trained models
print("\n[1/6] Saving trained models...")
for name, model in models.items():
    filename = f"fraud_detection_models/{name.replace(' ', '_').lower()}_model.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"   ✓ {name} saved")

# 2. Save scaler
print("\n[2/6] Saving scaler...")
with open('fraud_detection_models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("   ✓ Scaler saved")

# 3. Save feature names
print("\n[3/6] Saving feature names...")
with open('fraud_detection_models/feature_names.pkl', 'wb') as f:
    pickle.dump(X_train.columns.tolist(), f)
print("   ✓ Feature names saved")

# 4. Save model results
print("\n[4/6] Saving model results...")
results_df.to_csv('fraud_detection_models/model_comparison.csv', index=False)
print("   ✓ Results CSV saved")

# 5. Save best model info
print("\n[5/6] Saving best model metadata...")
best_model_info = {
    'model_name': best_name,
    'roc_auc': float(results_df.iloc[0]['ROC-AUC']),
    'precision': float(results_df.iloc[0]['Precision']),
    'recall': float(results_df.iloc[0]['Recall']),
    'f1_score': float(results_df.iloc[0]['F1-Score']),
    'accuracy': float(results_df.iloc[0]['Accuracy']),
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_size': len(df),
    'features_count': X.shape[1],
    'fraud_rate': float(df['isFraud'].mean())
}

with open('fraud_detection_models/best_model_info.json', 'w') as f:
    json.dump(best_model_info, f, indent=4)
print("   ✓ Metadata saved")

# 6. Save confusion matrices
print("\n[6/6] Saving confusion matrices...")
cm_data = {}
for name in models.keys():
    cm = confusion_matrix(y_test, predictions[name]['pred'])
    cm_data[name] = {
        'true_negatives': int(cm[0,0]),
        'false_positives': int(cm[0,1]),
        'false_negatives': int(cm[1,0]),
        'true_positives': int(cm[1,1])
    }

with open('fraud_detection_models/confusion_matrices.json', 'w') as f:
    json.dump(cm_data, f, indent=4)
print("   ✓ Confusion matrices saved")

print("\n" + "="*80)
print("✓ ALL ARTIFACTS SAVED TO: fraud_detection_models/")
print("="*80)

# List saved files
print("\nSaved files:")
!ls -lh fraud_detection_models/

In [ ]:
# ============================================================================
# ADVANCED VISUALIZATIONS (FIXED)
# ============================================================================
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("="*80)
print("GENERATING ADVANCED VISUALIZATIONS")
print("="*80)

# First, create confusion_matrices dictionary (was missing!)
print("\n[0/5] Creating confusion matrices for all models...")
confusion_matrices = {}
for name in models.keys():
    cm = confusion_matrix(y_test, predictions[name]['pred'])
    confusion_matrices[name] = cm
print("   ✓ Confusion matrices created")

# 1. ROC Curves for All Models
print("\n[1/5] ROC Curves...")
fig = go.Figure()

for name in models.keys():
    y_proba = predictions[name]['proba']
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)

    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        name=f'{name} (AUC={auc:.4f})',
        mode='lines',
        line=dict(width=2)
    ))

# Add diagonal
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    name='Random Classifier',
    mode='lines',
    line=dict(color='black', dash='dash', width=1)
))

fig.update_layout(
    title='ROC Curves - All Models',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=600,
    hovermode='closest'
)
fig.show()

# 2. Precision-Recall Curves
print("\n[2/5] Precision-Recall Curves...")
fig2 = go.Figure()

for name in models.keys():
    y_proba = predictions[name]['proba']
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba)
    avg_prec = average_precision_score(y_test, y_proba)

    fig2.add_trace(go.Scatter(
        x=recall_vals, y=precision_vals,
        name=f'{name} (AP={avg_prec:.4f})',
        mode='lines',
        line=dict(width=2)
    ))

fig2.update_layout(
    title='Precision-Recall Curves - All Models',
    xaxis_title='Recall',
    yaxis_title='Precision',
    height=600,
    hovermode='closest'
)
fig2.show()

# 3. Confusion Matrices Grid
print("\n[3/5] Confusion Matrices Grid...")
fig3 = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f'{name}' for name in models.keys()],
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}, {'type': 'heatmap'}]]
)

for idx, name in enumerate(models.keys(), 1):
    cm = confusion_matrices[name]

    fig3.add_trace(
        go.Heatmap(
            z=cm,
            x=['Legitimate', 'Fraud'],
            y=['Legitimate', 'Fraud'],
            colorscale='Blues',
            showscale=(idx==3),
            text=cm,
            texttemplate='%{text}',
            textfont={"size": 14}
        ),
        row=1, col=idx
    )

fig3.update_layout(
    title_text='Confusion Matrices - All Models',
    height=400
)
fig3.show()

# 4. Metrics Radar Chart
print("\n[4/5] Metrics Radar Chart...")
fig4 = go.Figure()

metrics_to_plot = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'ROC-AUC']

for name in models.keys():
    model_data = results_df[results_df['Model'] == name].iloc[0]
    values = [model_data[m] for m in metrics_to_plot]
    values.append(values[0])  # Close the polygon

    fig4.add_trace(go.Scatterpolar(
        r=values,
        theta=metrics_to_plot + [metrics_to_plot[0]],
        fill='toself',
        name=name
    ))

fig4.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0.9, 1.0])),
    title='Model Performance Radar Chart',
    height=600
)
fig4.show()

# 5. Feature Importance Comparison
print("\n[5/5] Feature Importance (Top 15)...")
if hasattr(best_model, 'feature_importances_'):
    feature_imp = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False).head(15)

    fig5 = px.bar(
        feature_imp,
        x='Importance',
        y='Feature',
        orientation='h',
        title=f'Top 15 Feature Importances - {best_name}',
        color='Importance',
        color_continuous_scale='Viridis'
    )
    fig5.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
    fig5.show()

print("\n✓ All visualizations generated!")

In [ ]:
# ============================================================================
# PRODUCTION-READY PREDICTION FUNCTION (FIXED)
# ============================================================================

def predict_fraud_with_explanation(transaction_data,
                                   provide_explanation=True,
                                   top_features=5):
    """
    Predict fraud probability with optional SHAP explanation

    Parameters:
    -----------
    transaction_data : dict or pd.DataFrame
        Transaction features (must match training features)
    provide_explanation : bool
        Whether to generate SHAP explanation
    top_features : int
        Number of top contributing features to show

    Returns:
    --------
    dict : Prediction results with explanation
    """

    # Convert to DataFrame if dict
    if isinstance(transaction_data, dict):
        transaction_df = pd.DataFrame([transaction_data])
    else:
        transaction_df = transaction_data.copy()

    # Ensure correct feature order
    transaction_df = transaction_df[X_train.columns]

    # Scale
    scaled_data = scaler.transform(transaction_df)
    scaled_df = pd.DataFrame(scaled_data, columns=X_train.columns)

    # Predict
    prediction = best_model.predict(scaled_df)[0]
    probability = best_model.predict_proba(scaled_df)[0]

    # Determine risk level
    fraud_prob = probability[1]
    if fraud_prob > 0.8:
        risk_level = "CRITICAL"
    elif fraud_prob > 0.5:
        risk_level = "HIGH"
    elif fraud_prob > 0.3:
        risk_level = "MEDIUM"
    else:
        risk_level = "LOW"

    result = {
        'prediction': 'FRAUD' if prediction == 1 else 'LEGITIMATE',
        'fraud_probability': float(fraud_prob),
        'legitimate_probability': float(probability[0]),
        'risk_level': risk_level,
        'confidence': float(max(probability)),
        'model_used': best_name
    }

    # Add explanation if requested
    if provide_explanation:
        shap_values = explainer.shap_values(scaled_df)
        if isinstance(shap_values, list):
            shap_values = shap_values[1]

        # Get top contributing features
        feature_contributions = pd.DataFrame({
            'feature': X_train.columns,
            'shap_value': shap_values[0],
            'abs_shap_value': np.abs(shap_values[0])
        }).sort_values('abs_shap_value', ascending=False)

        top_contrib = feature_contributions.head(top_features)[['feature', 'shap_value']].to_dict('records')

        result['explanation'] = {
            'top_contributing_features': top_contrib,
            'total_features_analyzed': len(X_train.columns)
        }

    return result

# Test the function
print("="*80)
print("TESTING PREDICTION FUNCTION")
print("="*80)

# Test with a fraud case - FIXED indexing
fraud_idx = y_test[y_test==1].index[0]
fraud_sample = X_test.loc[[fraud_idx]]  # Use .loc instead of .iloc
result_fraud = predict_fraud_with_explanation(fraud_sample, provide_explanation=True)

print("\n[TEST 1] Fraudulent Transaction:")
print(f"  Prediction: {result_fraud['prediction']}")
print(f"  Fraud Probability: {result_fraud['fraud_probability']:.2%}")
print(f"  Risk Level: {result_fraud['risk_level']}")
print(f"  Model: {result_fraud['model_used']}")
print("\n  Top Contributing Features:")
for i, feat in enumerate(result_fraud['explanation']['top_contributing_features'], 1):
    direction = "→ Fraud" if feat['shap_value'] > 0 else "→ Legit"
    print(f"    {i}. {feat['feature']:30s} {direction:12s} (impact: {abs(feat['shap_value']):.4f})")

# Test with legitimate case - FIXED indexing
legit_idx = y_test[y_test==0].index[0]
legit_sample = X_test.loc[[legit_idx]]  # Use .loc instead of .iloc
result_legit = predict_fraud_with_explanation(legit_sample, provide_explanation=True)

print("\n[TEST 2] Legitimate Transaction:")
print(f"  Prediction: {result_legit['prediction']}")
print(f"  Fraud Probability: {result_legit['fraud_probability']:.2%}")
print(f"  Risk Level: {result_legit['risk_level']}")

print("\n✓ Prediction function created and tested successfully!")

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("="*80)
print("CREATING INTERACTIVE PREDICTION DEMO")
print("="*80)

def create_fraud_detection_demo():
    """Interactive demo for fraud prediction"""

    output = widgets.Output()

    # Create button
    button = widgets.Button(
        description="🔍 Analyze Random Transaction",
        button_style='primary',
        layout=widgets.Layout(width='300px', height='50px')
    )

    # Transaction type filter
    filter_dropdown = widgets.Dropdown(
        options=['All', 'Fraudulent', 'Legitimate'],
        value='All',
        description='Filter:',
        style={'description_width': 'initial'}
    )

    def on_button_click(b):
        with output:
            clear_output()

            # Filter based on dropdown
            if filter_dropdown.value == 'Fraudulent':
                available_idx = y_test[y_test == 1].index
            elif filter_dropdown.value == 'Legitimate':
                available_idx = y_test[y_test == 0].index
            else:
                available_idx = y_test.index

            if len(available_idx) == 0:
                print("No transactions available")
                return

            # Get random transaction
            random_idx = np.random.choice(available_idx)
            transaction = X_test.loc[[random_idx]]
            actual_label = y_test.loc[random_idx]

            # Predict
            result = predict_fraud_with_explanation(transaction, provide_explanation=True, top_features=5)

            # Display results
            print("="*80)
            print(f"TRANSACTION ANALYSIS - ID: {random_idx}")
            print("="*80)

            # Status
            match_emoji = "✓" if (actual_label == 1 and result['prediction'] == 'FRAUD') or \
                               (actual_label == 0 and result['prediction'] == 'LEGITIMATE') else "✗"

            print(f"\nACTUAL:     {'🔴 Fraudulent' if actual_label == 1 else '🟢 Legitimate'}")
            print(f"PREDICTED:  {'🔴 ' + result['prediction']}")
            print(f"MATCH:      {match_emoji} {'Correct' if match_emoji == '✓' else 'Incorrect'}")

            print(f"\nRISK ASSESSMENT:")
            print(f"  Fraud Probability:  {result['fraud_probability']:.2%}")
            print(f"  Risk Level:         {result['risk_level']}")
            print(f"  Confidence:         {result['confidence']:.2%}")
            print(f"  Model Used:         {result['model_used']}")

            print(f"\nTOP 5 CONTRIBUTING FACTORS:")
            for i, feat in enumerate(result['explanation']['top_contributing_features'], 1):
                impact = "Increases Fraud Risk" if feat['shap_value'] > 0 else "Decreases Fraud Risk"
                bar_length = int(abs(feat['shap_value']) * 20)
                bar = "█" * bar_length
                print(f"  {i}. {feat['feature'][:35]:35s}")
                print(f"     {impact:25s} {bar} ({abs(feat['shap_value']):.4f})")

            print("\n" + "="*80)

            # Visualization
            fig = go.Figure()

            # Probability gauge
            fig.add_trace(go.Indicator(
                mode = "gauge+number+delta",
                value = result['fraud_probability'] * 100,
                title = {'text': "Fraud Probability (%)"},
                delta = {'reference': 50},
                gauge = {
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "darkred" if result['fraud_probability'] > 0.5 else "darkgreen"},
                    'steps': [
                        {'range': [0, 30], 'color': "lightgreen"},
                        {'range': [30, 50], 'color': "lightyellow"},
                        {'range': [50, 80], 'color': "orange"},
                        {'range': [80, 100], 'color': "red"}
                    ],
                    'threshold': {
                        'line': {'color': "black", 'width': 4},
                        'thickness': 0.75,
                        'value': 50
                    }
                }
            ))

            fig.update_layout(height=400)
            fig.show()

    button.on_click(on_button_click)

    # Display interface
    display(HTML("<h2>🎯 Fraud Detection Interactive Demo</h2>"))
    display(HTML("<p>Click the button to analyze a random transaction from the test set</p>"))
    display(filter_dropdown)
    display(button)
    display(output)

# Launch demo
create_fraud_detection_demo()

print("\n✓ Interactive demo created!")
print("Click the button above to test predictions!")